In [1]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import cv2
import os

In [2]:
#!wget http://images.cocodataset.org/val2017/000000439715.jpg -O input.jpg
im = cv2.imread("./image24.jpg")


In [3]:
cfg = get_cfg()
cfg.merge_from_file("./detectron2/model_zoo/configs/COCO-Detection/retinanet_R_101_FPN_3x.yaml")
cfg.MODEL.RETINANET.NMS_THRESH_TEST = 0.5
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.5
cfg.MODEL.RETINANET.NUM_CLASSES = 1
cfg.DATASETS.TEST = ('tomato_data/test',)
#cfg.MODEL.WEIGHTS = "model_final_59f53c.pkl"
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

Loading config ./detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.
Config './detectron2/model_zoo/configs/COCO-Detection/retinanet_R_101_FPN_3x.yaml' has no VERSION. Assuming it to be compatible with latest v2.


In [4]:
import torch
torch.device("cuda:0")

device(type='cuda', index=0)

In [5]:
# Create predictor
predictor = DefaultPredictor(cfg)

# Make prediction
#outputs = predictor(im)

In [6]:
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2.imshow('s',v.get_image()[:, :, ::-1])
cv2.imwrite('result3.jpg', v.get_image()[:, :, ::-1])
cv2.waitKey(0)
cv2.destroyAllWindows()

TypeError: 'NoneType' object is not subscriptable

In [7]:
import os
import numpy as np
import json
from detectron2.structures import BoxMode
import itertools
import cv2
import pandas as pd
import random
def get_tomato_dicts(csv_file, img_dir):
    df = pd.read_csv(csv_file)
    df['filename'] = df['filename'].map(lambda x: img_dir+x)

    classes = ['tomato']
    df['class'] = df['class'].map(lambda x: 'tomato')
    df['class_int'] = df['class'].map(lambda x: classes.index(x))

    dataset_dicts = []
    for filename in df['filename'].unique().tolist():
        record = {}
        
        image = cv2.imread(filename)
        
        height, width, _ = image.shape
        
        record["file_name"] = filename
        record["height"] = height
        record["width"] = width

        objs = []
        for index, row in df[(df['filename']==filename)].iterrows():
          obj= {
              'bbox': [row['xmin'], row['ymin'], row['xmax'], row['ymax']],
              'bbox_mode': BoxMode.XYXY_ABS,
              'category_id': row['class_int'],
              "iscrowd": 0
          }
          objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [29]:
treshhold = 0.7
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
#cfg.MODEL.RETINANET.NMS_THRESH_TEST = 0.1
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = treshhold
cfg.MODEL.RETINANET.NUM_CLASSES = 1 # set the testing threshold for this model
cfg.DATASETS.TEST = ('tomato_data/test', )
cfg.MODEL.RETINANET.IOU_THRESHOLDS = [0.7, 0.8]
predictor = DefaultPredictor(cfg)
tomato_metadata = MetadataCatalog.get('tomato/test').set(thing_classes=['tomato'])
df_test = pd.read_csv('tomato/test_labels.csv')
dataset_dicts = get_tomato_dicts('tomato/test_labels.csv', 'tomato/test/')

import os
if not os.path.exists('test_results/bboxs/' + str(treshhold) + '/'):
    os.makedirs('test_results/bboxs/' + str(treshhold)+ '/')

i = 0
for d in dataset_dicts:    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    print(outputs)
    v = Visualizer(im[:, :, ::-1], metadata=tomato_metadata, scale=0.8)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imwrite('./test_results/images/result{}.jpg'.format(i), v.get_image()[:, :, ::-1])
    i += 1

{'instances': Instances(num_instances=10, image_height=3000, image_width=3000, fields=[pred_boxes = Boxes(tensor([[ 822.1724, 1090.6899, 1650.5684, 1941.9185],
        [ 636.6909, 1919.5466, 1422.2383, 2763.4839],
        [ 785.5433,  187.1197, 1760.8262, 1111.6777],
        [1867.0598, 2648.9470, 2044.3925, 2887.7925],
        [1354.6057, 1662.7881, 2398.0920, 2669.3535],
        [1746.5238,    0.0000, 2971.0732,  764.3489],
        [   0.0000, 1248.4828,  964.8246, 2265.4463],
        [1550.9573,  719.1326, 2619.1831, 1683.5706],
        [1280.7455, 2415.9829, 1676.8624, 2700.7742],
        [1857.0686, 2907.8088, 2105.3484, 2997.7402]], device='cuda:0')), scores = tensor([0.9998, 0.9993, 0.9993, 0.9991, 0.9989, 0.9988, 0.9983, 0.9979, 0.9913,
        0.7494], device='cuda:0'), pred_classes = tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'), ])}
{'instances': Instances(num_instances=7, image_height=1400, image_width=1400, fields=[pred_boxes = Boxes(tensor([[6.6213e+02, 3.5595e+

{'instances': Instances(num_instances=5, image_height=3264, image_width=4928, fields=[pred_boxes = Boxes(tensor([[3230.5776, 1475.5704, 3548.9236, 1837.7772],
        [3572.2153, 1770.5663, 3905.4097, 2141.4270],
        [ 301.7585, 2865.6313,  678.5829, 3257.8105],
        [ 590.9800, 2510.0845, 1212.4772, 3105.3496],
        [ 368.9145, 2583.5298,  609.1479, 2826.7971]], device='cuda:0')), scores = tensor([1.0000, 0.9999, 0.9998, 0.9996, 0.7704], device='cuda:0'), pred_classes = tensor([0, 0, 0, 0, 0], device='cuda:0'), ])}
{'instances': Instances(num_instances=5, image_height=3264, image_width=4928, fields=[pred_boxes = Boxes(tensor([[ 805.9106, 2632.5266, 1288.0310, 3084.6560],
        [1225.3073, 2866.1650, 1583.6256, 3239.4866],
        [4585.0605, 2130.7966, 4928.0000, 2517.2527],
        [  32.0763, 1691.9136,  299.4386, 1940.9071],
        [2939.1526, 1874.6578, 3062.0493, 1991.1282]], device='cuda:0')), scores = tensor([1.0000, 1.0000, 0.9999, 0.9997, 0.8769], device='cuda:0'

{'instances': Instances(num_instances=7, image_height=3000, image_width=3000, fields=[pred_boxes = Boxes(tensor([[ 829.2869, 2102.8701, 1658.9297, 2956.2964],
        [ 779.0335, 1200.9257, 1753.6061, 2140.7922],
        [1753.5537,  533.4396, 2993.0317, 1790.4929],
        [   9.8633, 2254.2314,  939.4733, 2983.3750],
        [1535.9193, 1741.1865, 2621.5449, 2698.6821],
        [1361.3992, 2685.8826, 2362.2271, 2987.4463],
        [ 936.6653, 2893.7996, 1353.2609, 2996.7871]], device='cuda:0')), scores = tensor([0.9997, 0.9996, 0.9995, 0.9991, 0.9982, 0.8431, 0.8162],
       device='cuda:0'), pred_classes = tensor([0, 0, 0, 0, 0, 0, 0], device='cuda:0'), ])}
{'instances': Instances(num_instances=6, image_height=3000, image_width=3000, fields=[pred_boxes = Boxes(tensor([[ 303.5148, 1637.0757, 1536.5071, 2938.8535],
        [1460.8210, 1661.4458, 2887.0146, 3000.0000],
        [1593.7295,  322.4517, 2998.4453, 1757.4855],
        [ 360.3438,    0.0000, 1701.9260,  683.2694],
        [ 